<a href="https://colab.research.google.com/github/josepeon/python_dad_class/blob/main/text_intro_to_rnn_dele.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification with Neural Networks

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

In [44]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#### Classifying Spam

In [45]:
#read in data
spam = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/refs/heads/master/sms_spam.csv')

In [46]:
#take a peek
spam.head()

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [47]:
#create a tokenizer
tokenizer = Tokenizer(num_words = 500)

In [48]:
#fit the tokenizer -- learns the vocabulary
tokenizer.fit_on_texts(spam['text'].values)

In [49]:
#look at tokenizer
tokenizer.num_words

500

In [50]:
#create document term matrix (binarized)
dtm = tokenizer.texts_to_matrix(spam['text'].values)

In [51]:
#take a peek
dtm

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [52]:
tokenizer.index_word[2]

'to'

In [53]:
spam['text'][2]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [54]:
[tokenizer.index_word[i] for i in range(1, 500)]

['i',
 'to',
 'you',
 'a',
 'the',
 'u',
 'and',
 'in',
 'is',
 'me',
 'my',
 'for',
 'your',
 'it',
 'of',
 'call',
 'have',
 'on',
 '2',
 'that',
 'now',
 'are',
 'so',
 'but',
 'not',
 'or',
 'do',
 'can',
 'at',
 "i'm",
 'ur',
 'get',
 'will',
 'if',
 'be',
 'with',
 'just',
 'no',
 'we',
 'this',
 '4',
 'gt',
 'lt',
 'up',
 'when',
 'ok',
 'free',
 'from',
 'go',
 'how',
 'all',
 'out',
 'what',
 'know',
 'like',
 'good',
 'then',
 'got',
 'come',
 'was',
 'its',
 'am',
 'time',
 'only',
 'day',
 'love',
 'there',
 'send',
 'he',
 'want',
 'text',
 'as',
 'txt',
 'one',
 'going',
 'by',
 'ü',
 "i'll",
 'need',
 'home',
 'about',
 'r',
 'lor',
 'sorry',
 'stop',
 'still',
 'see',
 'n',
 'back',
 'today',
 'da',
 'our',
 'dont',
 'reply',
 'k',
 "don't",
 'she',
 'mobile',
 'take',
 'hi',
 'tell',
 'new',
 'please',
 'later',
 'her',
 'pls',
 'any',
 'think',
 'been',
 'they',
 'phone',
 'here',
 'week',
 'did',
 'dear',
 'some',
 'well',
 'has',
 '1',
 'night',
 'much',
 'd',
 'gre

In [55]:
class TextDataset(Dataset):
  def __init__(self, X, y):
    super().__init__()
    self.x = torch.tensor(X, dtype = torch.float)
    self.y = torch.tensor(y, dtype = torch.float)

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [56]:
X = dtm
y = np.where(spam['type'] == 'ham', 0, 1)

In [57]:
Xt = torch.tensor(X, dtype = torch.float32)
yt = torch.tensor(y, dtype = torch.float32)

In [58]:
from torch.utils.data import TensorDataset, DataLoader

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
X_train, X_test, y_train, y_test = train_test_split(Xt, yt, test_size=.2)

In [61]:
X_train

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])

In [62]:
#create data class
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

/tmp/ipython-input-3856549849.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(X, dtype = torch.float)
/tmp/ipython-input-3856549849.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype = torch.float)


In [63]:
#dataset and loader -- making batches of our bigger dataset
trainloader = DataLoader(train_dataset, batch_size = 32)
#dataset and loader
testloader = DataLoader(test_dataset, batch_size = 32)

In [64]:
model = nn.Sequential(nn.Linear(in_features=500, out_features=1000),
                      nn.ReLU(),
                      nn.Linear(1000, 100),
                      nn.ReLU(),
                      nn.Linear(100, 1),
                      nn.Sigmoid()
                      )

In [65]:
model(Xt)

tensor([[0.4973],
        [0.4919],
        [0.4919],
        ...,
        [0.4931],
        [0.4908],
        [0.4927]], grad_fn=<SigmoidBackward0>)

In [66]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [67]:
from tqdm import tqdm

In [68]:
#keep track of the losses
losses = []
#train it for 20 epochs
for epoch in tqdm(range(20)):
  #iterate over the batches
  for x, y in trainloader:
    #feeds data into the model
    yhat = model(x)
    #evaluate the predictions
    loss =loss_fn(yhat, y.unsqueeze(1))
    #update the weights/params
    optimizer.zero_grad() #pytorch house cleaning
    loss.backward() #pass info backward
    optimizer.step() #step toward less loss
    losses.append(loss.item()) #tracking the loss

100%|██████████| 20/20 [00:34<00:00,  1.73s/it]


In [69]:
train_predictions = model(X_train)
ytrain_preds = torch.where(train_predictions> .5, 1, 0)

In [70]:
torch.sum(ytrain_preds.squeeze(1) == y_train)/len(y_train)

tensor(0.9993)

In [71]:
ytest_preds = torch.where(model(X_test) > .5, 1, 0)
torch.sum(ytest_preds.squeeze(1) == y_test)/len(y_test)

tensor(0.9892)

In [72]:
#loss and optimizer
class TextModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.lin1 = nn.Linear(in_features = 500, out_features = 100)
    self.lin2 = nn.Linear(100, 100)
    self.lin3 = nn.Linear(100, 1)
    self.sigmoid = nn.Sigmoid()
    self.act = nn.ReLU()

  def forward(self, x):
    x = self.act(self.lin1(x))
    x = self.act(self.lin2(x))
    return self.sigmoid(self.lin3(x))




In [73]:
#training function
model = TextModel()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [74]:
#torch.save(model, 'textmodel.pt')

In [75]:
from tqdm import tqdm

In [76]:
#evaluate
for epoch in tqdm(range(100)):
  losses = 0
  for x,y in trainloader:
    yhat = model(x)
    y = y.reshape(-1, 1)
    loss = loss_fn(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses += loss.item()
  if epoch % 10 == 0:
    print(f'Epoch {epoch} Loss: {losses}')

  1%|          | 1/100 [00:00<00:30,  3.28it/s]

Epoch 0 Loss: 17.46782467002049


 11%|█         | 11/100 [00:03<00:31,  2.86it/s]

Epoch 10 Loss: 0.2164153954182666


 21%|██        | 21/100 [00:07<00:32,  2.44it/s]

...

 81%|████████  | 81/100 [00:34<00:09,  2.01it/s]

Epoch 80 Loss: 0.14997812902651736


 91%|█████████ | 91/100 [00:38<00:03,  2.41it/s]

Epoch 90 Loss: 0.15083457416988288


100%|██████████| 100/100 [00:42<00:00,  2.37it/s]


In [77]:
Xt = torch.tensor(X_test, dtype = torch.float)

/tmp/ipython-input-747476605.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xt = torch.tensor(X_test, dtype = torch.float)


In [78]:

output = model(Xt) #model predictions

In [79]:
output

tensor([[2.1505e-20],
        [6.2030e-07],
        [7.6829e-35],
        ...,
        [0.0000e+00],
        [7.6035e-28],
        [2.3761e-32]], grad_fn=<SigmoidBackward0>)

In [80]:
#Converting probabilities to prediction
preds = np.where(np.array(output.detach()) >= .5, 1, 0)

In [81]:
preds.shape

(1115, 1)

In [82]:
y = np.where(spam['type'] == 'ham', 0, 1)

In [83]:
sum(preds[:, 0] == y_test)/len(y_test)

tensor(0.9892)

In [84]:
1 - sum(y_test)/len(y_test)

tensor(0.8762)

### Basic RNN

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Recurrent_neural_network_unfold.svg/440px-Recurrent_neural_network_unfold.svg.png)

In [85]:
#create sequences
sequences = tokenizer.texts_to_sequences(spam['text'].values)

In [86]:
#look at first sequence
sequences[0]

[49, 471, 64, 8, 88, 123, 351, 148, 67, 58, 145]

In [87]:
X_train[0]

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [88]:
#compare to text
spam['text'].values[1]

'Ok lar... Joking wif u oni...'

In [89]:
#pad and make all same length
sequences = pad_sequences(sequences, maxlen=100)

In [90]:
#examine results
sequences[1].shape

(100,)

In [91]:
sequences[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,  46, 336, 472,   6], dtype=int32)

In [92]:
#example rnn
rnn = nn.RNN(input_size = 100,
             hidden_size = 30,
             num_layers = 1,
             batch_first = True)

In [93]:
#pass data through
sample_sequence = torch.tensor(sequences[1],
                               dtype = torch.float,
                               ).reshape(1, -1)
sample_sequence.shape

torch.Size([1, 100])

In [94]:
#output
output, hidden = rnn(sample_sequence)

In [95]:
#hidden
hidden.shape

torch.Size([1, 30])

In [96]:
#linear layer
output.shape

torch.Size([1, 30])

In [97]:
#pass through linear
lin1 = nn.Linear(in_features = 30, out_features = 1)

In [98]:
lin1(output)

tensor([[0.3807]], grad_fn=<AddmmBackward0>)

In [99]:
for x, y in trainloader:
  print(x.shape)
  break

torch.Size([32, 500])


In [100]:
class TextDataset(Dataset):
  def __init__(self, X, y):
    super().__init__()
    self.x = torch.tensor(X, dtype = torch.float)
    self.y = torch.tensor(y, dtype = torch.float)

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [101]:
#class
class BasicRNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = nn.RNN(input_size = 100,
                    hidden_size = 50,
                    num_layers = 3,
                    batch_first = True)
    self.lin1 = nn.Linear(in_features = 50, out_features=1000)
    self.lin2 = nn.Linear(1000, 100)
    self.lin3 = nn.Linear(100, 1)
    self.act = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x, _ = self.rnn(x) #extracting important information
    x = self.act(self.lin1(x)) #multilayer perceptron -- to predict
    x = self.act(self.lin2(x))
    x = self.sigmoid(self.lin3(x))
    return x


In [102]:
#data
X = sequences
y = np.where(spam['type'] == 'spam', 1, 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)
traindata = TextDataset(X_train, y_train)
trainloader = DataLoader(traindata, batch_size = 32)

In [103]:
#optimizer and loss
model = BasicRNN()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [104]:
#train
for epoch in tqdm(range(100)):
  losses = 0
  for x,y in trainloader:
    yhat = model(x)
    y = y.reshape(-1, 1)
    loss = loss_fn(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses += loss.item()
  if epoch % 10 == 0:
    print(f'Epoch {epoch} Loss: {losses}')

  1%|          | 1/100 [00:01<03:04,  1.86s/it]

Epoch 0 Loss: 53.93538190424442


 11%|█         | 11/100 [00:23<03:09,  2.13s/it]

Epoch 10 Loss: 48.80645204335451


 21%|██        | 21/100 [00:45<02:55,  2.23s/it]

...

 81%|████████  | 81/100 [03:02<00:43,  2.30s/it]

Epoch 80 Loss: 55.593467220664024


 91%|█████████ | 91/100 [03:25<00:20,  2.31s/it]

Epoch 90 Loss: 55.5934783667326


100%|██████████| 100/100 [03:46<00:00,  2.27s/it]


In [105]:
Xt = torch.tensor(X_test, dtype = torch.float)

In [106]:
output = model(Xt)

In [107]:
preds = np.where(np.array(output.detach()) >= .5, 1, 0)

In [108]:
#preds = output.argmax(axis = 1)

In [109]:
y_test

array([0, 1, 0, ..., 0, 0, 0])

In [110]:
# y = np.where(spam['type'] == 'ham', 0, 1)

In [111]:
# y.shape

In [112]:
sum(preds.reshape(1115,) == y_test)/len(y_test)

np.float64(0.874439461883408)

#### LSTM

In [113]:
# nn.LSTM()
class BasicLSTM(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = nn.LSTM(input_size = 100,
                    hidden_size = 100,
                    num_layers = 1,
                    batch_first = True)

    self.lin1 = nn.Linear(in_features = 100, out_features=100)
    self.lin2 = nn.Linear(in_features = 100, out_features = 1)
    self.act = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.act(self.lin1(x))
    x = self.lin2(x)
    return self.sigmoid(x)

In [114]:
model = BasicLSTM()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [115]:
#train
for epoch in range(10):
  losses = 0
  for x,y in trainloader:
    yhat = model(x)
    y = y.reshape(-1, 1)
    loss = loss_fn(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses += loss.item()
  if epoch % 10 == 0:
    print(f'Epoch {epoch} Loss: {losses}')

Epoch 0 Loss: 49.46761464327574


In [116]:
Xt = torch.tensor(X_test, dtype = torch.float)
output = model(Xt)
preds = np.where(np.array(output.detach()) >= .5, 1, 0)
sum(preds[:, 0] == y_test)/len(y_test)

np.float64(0.874439461883408)

In [117]:
#pad and make all same length
sequences = pad_sequences(sequences, maxlen=30)

In [118]:
sequences[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,  49, 471,  64,   8,  88, 123, 351,
       148,  67,  58, 145], dtype=int32)

In [119]:
X = sequences
y = np.where(spam['type'] == 'spam', 1, 0)
data = TextDataset(X, y)
loader = DataLoader(data, batch_size = 32)

In [120]:
class RNN2(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = nn.GRU(input_size = 30,
                    hidden_size = 30,
                    num_layers = 2,
                    batch_first = True)

    self.lin1 = nn.Linear(in_features = 30, out_features=100)
    self.lin2 = nn.Linear(in_features = 100, out_features = 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.lin1(x)
    x = self.lin2(x)
    return self.sigmoid(x)

In [121]:
model = RNN2()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [122]:
#train
for epoch in range(100):
  losses = 0
  for x,y in loader:
    yhat = model(x)
    y = y.reshape(-1, 1)
    loss = loss_fn(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses += loss.item()
  if epoch % 10 == 0:
    print(f'Epoch {epoch} Loss: {losses}')

Epoch 0 Loss: 67.75759781897068
Epoch 10 Loss: 59.209461092948914
Epoch 20 Loss: 59.173088788986206
Epoch 30 Loss: 59.83496733009815
Epoch 40 Loss: 59.70289013534784
Epoch 50 Loss: 58.032845586538315
Epoch 60 Loss: 55.7393634095788
Epoch 70 Loss: 56.244490049779415
Epoch 80 Loss: 56.146597906947136
Epoch 90 Loss: 57.155791848897934


In [123]:
Xt = torch.tensor(sequences, dtype = torch.float)
output = model(Xt)
preds = np.where(np.array(output.detach()) >= .5, 1, 0)
y = np.where(spam['type'] == 'ham', 0, 1)
sum(preds[:, 0] == y)/len(y)

np.float64(0.8670613562970937)